<a href="https://colab.research.google.com/github/navidadkhah/Fine-Tuning-LLMs/blob/main/Dataset/Create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing requirement libraries

In [ ]:
!pip install mutmut
!pip install astor
!pip install selenium
!pip install webdriver-manager
!pip install selenium webdriver-manager
!apt-get update
!apt-get install -y firefox
!pip install selenium webdriver-manager

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,159 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,318 kB]
Get:14 http

In [ ]:
# Import required modules
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager

# Set up Firefox options
firefox_options = Options()
firefox_options.add_argument("--headless")  # Run in headless mode
firefox_options.add_argument("--no-sandbox")
firefox_options.add_argument("--disable-dev-shm-usage")

# Set up the Firefox WebDriver
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=firefox_options)

# Now you can navigate to a website
driver.get('https://www.example.com')

# Print the title of the page
print(driver.title)

# Close the driver when done
driver.quit()


WebDriverException: Message: Process unexpectedly closed with status 1


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager

# Setup Selenium WebDriver
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()))

# Function to automate browser and scrape code
def selenium_scrape(contest_id, submission_id):
    url = f"https://codeforces.com/contest/{contest_id}/submission/{submission_id}"

    # Open the webpage
    driver.get(url)

    # Find the element that contains the source code
    code_element = driver.find_element('xpath', '//pre[@class="prettyprint"]')

    if code_element:
        source_code = code_element.text
        print("Source code extracted:")
        print(source_code)
    else:
        print("Couldn't find the source code on the page.")

    # Close the browser
    driver.quit()

# Example usage:
selenium_scrape(566, 12345678)


SessionNotCreatedException: Message: Expected browser binary location, but unable to find binary in default location, no 'moz:firefoxOptions.binary' capability provided, and no binary flag set on the command line


In [ ]:
import requests
import numpy as np
import ast    # To convert code to AST
import astor  # To convert AST back to code
import random  # To select mutations randomly

# Mutanting code
In this section, we mutant code using the AST library. Changing operations in the code and randomly replace with AST operations.
<br>
In the output, we expect a mutant code and a line to tell us which line has changed.

In [ ]:
# Define a function to mutate and print code
def mutate_code(code_str, num_mutations):
    # Parse the string into an AST
    tree = ast.parse(code_str)

    # List of possible binary operations for mutation
    operations = [ast.Add, ast.Sub, ast.Mult, ast.Div, ast.Mod, ast.Pow,
                  ast.FloorDiv, ast.LShift, ast.RShift, ast.BitOr, ast.BitXor,
                  ast.BitAnd, ast.MatMult]

    # Split the original code into lines
    original_lines = code_str.strip().split('\n')

    for i in range(0, num_mutations):
        # Apply a random mutation to the AST
        for node in ast.walk(tree):
            if isinstance(node, ast.BinOp):
                # Randomly choose a new operator
                new_op = random.choice(operations)()
                node.op = new_op

        # Convert the mutated AST back to a string
        mutated_code = astor.to_source(tree).strip()
        mutated_lines = mutated_code.split('\n')

        # Print the mutated code
        print(f"\n--- Mutation {i + 1} ---")
        print(mutated_code)


        # Identify which lines have changed
        changed_lines = []
        line = 1;
        for original, mutated in zip(original_lines, mutated_lines):
            if original != mutated:
              print(f"Line {line} has changed")
              changed_lines.append(mutated)
            line += 1

        # Output changed lines
        if changed_lines:
            print("\n--- Changed Lines ---")
            for line in changed_lines:
                print(line)
        else:
            print("No changes detected.")

        # Reset the tree by re-parsing the original code
        tree = ast.parse(code_str)

ModuleNotFoundError: No module named 'astor'

In [ ]:
# Example usage: Pass code as a string
original_code ="""
def add(a, b):
    if a > 0:
        return a - b
    else:
        return a + b
"""

mutate_code(original_code, 2)


--- Mutation 1 ---
def add(a, b):
    if a > 0:
        return a ** b
    else:
        return a ^ b
Line 3 has changed
Line 5 has changed

--- Changed Lines ---
        return a ** b
        return a ^ b

--- Mutation 2 ---
def add(a, b):
    if a > 0:
        return a ^ b
    else:
        return a - b
Line 3 has changed
Line 5 has changed

--- Changed Lines ---
        return a ^ b
        return a - b


# Create Dataset

In [ ]:
# In this cell, we get datas from below url. datas contain number of available contests
# Usefull variables:
#     last_contest_id : keeps id of the last contest
#     number_of_contests : keeps number of FINISHED contests
#     contest_IDs : keeps the ID contests


url = "https://codeforces.com/api/contest.list"

response = requests.get(url)
data = response.json()

if data['status'] == "OK":
    contests = data['result']

    # Filter and print only Python submissions
    last_contest_id = 0
    not_started_yet = 0
    contest_IDs = []
    for contest in contests:
      if contest['phase'] == 'BEFORE':
        not_started_yet += 1
      else:
        if last_contest_id == 0:
          last_contest_id = contest['id']
        contest_IDs.append(contest['id'])


    number_of_contests = len(contests) - not_started_yet

    contest_IDs.sort()
    print(f"Number of available contests: {number_of_contests}\nID of the last contest: {last_contest_id}")

else:
    print("Error:", data['comment'])

Number of available contests: 1915
ID of the last contest: 2021


In [ ]:
# Now, we use this Function to get the best users from contests
# Usefull variables
#     rated_list_handle : keeps all the best handles

def get_contest_users(contest_id, count, rated_list_handle):

    url = f"https://codeforces.com/api/contest.standings?contestId={contest_id}&from=1&count={count}"

    # Make the GET request to Codeforces API
    response = requests.get(url)
    data = response.json()

    # Check if the API call was successful
    if data['status'] == 'OK':
        standings = data['result']['rows']

        # Extract and print user handles
        for row in standings:
            handle = row['party']['members'][0]['handle']
            rank = row['rank']
            if handle not in rated_list_handle:
              rated_list_handle.append(handle)
            # print(f"Rank: {rank}, User: {handle}")
    else:
        print("Error:", data['comment'])

# Example: Get the first 10 users from contest 566
number_of_rated = 5
rated_list_handle = []
for index in range(len(contest_IDs), 0, -1):
    get_contest_users(index, number_of_rated, rated_list_handle)
print(f"Number of top rated contestor of all time: {len(rated_list_handle)}")
print(f"Some example:\n{rated_list_handle[0:10]}")

Error: contestId: Contest with id 1908 not found
Error: contestId: Contest with id 1897 not found
Error: contestId: Contest with id 1892 not found
Error: contestId: Contest with id 1880 not found
Error: contestId: Contest with id 1871 not found
Error: contestId: Contest with id 1865 not found
Error: contestId: Contest with id 1803 not found
Error: contestId: Contest with id 1757 not found
Error: contestId: Contest with id 1756 not found
Error: contestId: Contest with id 1745 not found
Error: contestId: Contest with id 1727 not found
Error: contestId: Contest with id 1683 not found
Error: contestId: Contest with id 1664 not found
Error: contestId: Contest with id 1655 not found
Error: contestId: Contest with id 1653 not found
Error: contestId: Contest with id 1645 not found
Error: contestId: Contest with id 1643 not found
Error: contestId: Contest with id 1640 not found
Error: contestId: Contest with id 1636 not found
Error: contestId: Contest with id 1597 not found
Error: contestId: Co

In [ ]:
import json

# Load the JSON data from the file
with open('submissions.json', 'r') as json_file:
    submissions_data = json.load(json_file)

# Iterate through the submissions and print their details
for submission in submissions_data["submissions"]:
    submission_id = submission["submission_id"]
    contest_id = submission["contest_id"]
    user_handle = submission["user_handle"]

    print(f"Submission ID: {submission_id}, Contest ID: {contest_id}, User Handle: {user_handle}")


Submission ID: 123456, Contest ID: 567, User Handle: example_user_1
Submission ID: 123457, Contest ID: 568, User Handle: example_user_2
Submission ID: 1, Contest ID: 8, User Handle: nawid


In [ ]:
import json

# Function to add a submission
def add_submission(contest_id, submission_id, problem_name, user_handle):
    submission = {
        "contest_id": contest_id,
        "submission_id": submission_id,
        "problem_name": problem_name,
        "user_handle": user_handle
    }
    submissions_data["submissions"].append(submission)

def find_python_submission(handle):
    # API URL to get submissions for the user
    url = f"https://codeforces.com/api/user.status?handle={handle}&from=1&count=10000"

    # Make the GET request to Codeforces API
    response = requests.get(url)
    data = response.json()

    # Check if the API call was successful
    if data['status'] == 'OK':
        submissions = data['result']

        # Filter and print only Python submissions
        index = 1
        number_of_pythons = 0
        for submission in submissions:
            if 'Python' in submission['programmingLanguage'] and submission['verdict'] == "OK":  # Check if the language is Python
              try:
                submission_id = submission['id']
                contest_id = submission['contestId']
                problem_name = submission['problem']['name']
              except:
                print("An exception occurred")

              #print(f"The index is {index}, Contest id: {contest_id}, Problem: {problem_name}, Language: {programming_language}, Verdict: {verdict}, Submission ID: {submission_id}")
              number_of_pythons += 1

            index += 1
        if number_of_pythons != 0:
           #print(f"The total numbers: {number_of_pythons}")
           add_submission(contest_id, submission_id, problem_name, handle)


    else:
        print("Error:", data['comment'])


submissions_data = {
    "submissions": []
}

for index in rated_list_handle:
   find_python_submission(index)

with open('submissions.json', 'w') as json_file:
    json.dump(submissions_data, json_file, indent=4)

# Print the JSON structure
print(json.dumps(submissions_data, indent=4))


An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred


UnboundLocalError: local variable 'contest_id' referenced before assignment

In [ ]:
url = f"https://codeforces.com/api/user.status?handle=244mhq&from=1&count=10000"

        # Make the GET request to Codeforces API
response = requests.get(url)
data = response.json()

        # Check if the API call was successful
if data['status'] == 'OK':
  submissions = data['result']

            # Filter and print only Python submissions
  index = 1
  number_of_pythons = 0
  for submission in submissions:
    if 'Python' in submission['programmingLanguage'] and submission['verdict'] == "OK":  # Check if the language is Python

      submission_id = submission['id']
      try:
        contest_id = submission['contestId']
      except:
        print("An exception occurred")
      problem_name = submission['problem']['name']
      programming_language = submission['programmingLanguage']
      verdict = submission['verdict']

                    #print(f"The index is {index}, Contest id: {contest_id}, Problem: {problem_name}, Language: {programming_language}, Verdict: {verdict}, Submission ID: {submission_id}")
      number_of_pythons += 1

    index += 1
  if number_of_pythons != 0:
              print(f"The total numbers: {number_of_pythons}")

else:
            print("Error:", data['comment'])


An exception occurred
The total numbers: 10
